# Java concurrency in practice

## Chapter - 4
## Composing Objects

## 4.1 Designing a thread-safeclass

## 4.2 Instance confinement

## 4.3 Delegating thread safety

## 4.4 Adding functionality to existing thread-safe classes 

## 4.5 Documenting synchronization policies